# IMPORTS

In [ ]:
#libs
import pandas as pd
import numpy as np
import folium
import streamlit as st
from folium.plugins import MarkerCluster
import plotly.express as px
from datetime import datetime
import geopandas
from streamlit_folium import folium_static

## Helper Functions

In [ ]:
#functions    
    
    def streamlit_settings():
        st.set_page_config(layout='wide')
        st.cache(allow_output_mutation=True)
        return None

    def get_data(path):
        data = pd.read_csv(path)
        return data

    st.cache(allow_output_mutation=True)
    def get_geofile(url):
        geofile = geopandas.read_file(url)
        return geofile

    def set_feature(data):
        #add new features
        data['price_m2'] = data['price']/data['sqft_lot']
        return data

    def overview_data(data):
        #Data Overview
        f_attributes = st.sidebar.multiselect('Enter columns', data.columns)
        f_zipcode = st.sidebar.multiselect('Enter zipcode',
                                           data['zipcode'].unique())
        st.title('Data Overview')

        if (f_zipcode != []) & (f_attributes != []):
            data = data.loc[data['zipcode'].isin(f_zipcode), f_attributes]

        elif (f_zipcode != []) & (f_attributes == []):
            data = data.loc[data['zipcode'].isin(f_zipcode), :]

        elif (f_zipcode == []) & (f_attributes != []):
            data = data.loc[:, f_attributes]

        else:
            data = data.copy()

        st.dataframe(data)

        c1, c2 = st.beta_columns((2, 1))

        # average metrics:
        df1 = data[['id', 'zipcode']].groupby('zipcode').count().reset_index()
        df2 = data[['price', 'zipcode']].groupby('zipcode').mean().reset_index()
        df3 = data[['sqft_living', 'zipcode']].groupby('zipcode').mean().reset_index()
        df4 = data[['price_m2', 'zipcode']].groupby('zipcode').mean().reset_index()

        # merge
        m1 = pd.merge(df1, df2, on='zipcode', how='inner')
        m2 = pd.merge(m1, df3, on='zipcode', how='inner')
        m3 = pd.merge(m2, df4, on='zipcode', how='inner')

        m3.columns = ['zipcode', 'total houses', 'price', 'sqft living', 'price/m2']

        c1.header('Average Values')
        c1.dataframe(m3, height=600)

        # descriptive stats
        num_attributes = data.select_dtypes(['int64', 'float64'])
        media = pd.DataFrame(num_attributes.apply(np.mean))
        mediana = pd.DataFrame(num_attributes.apply(np.median))
        std = pd.DataFrame(num_attributes.apply(np.std))
        max_ = pd.DataFrame(num_attributes.apply(np.max))
        min_ = pd.DataFrame(num_attributes.apply(np.min))

        df1 = pd.concat([max_, min_, media, mediana, std], axis=1).reset_index()

        df1.columns = ['attributes', 'max', 'min', 'mean', 'median', 'std']

        c2.header('Descriptive Statistics')
        c2.dataframe(df1, height=800)

        st.write(data.head())
        # attributes => colunas
        # zipcode => linhas

        st.write(f_attributes)
        st.write(f_zipcode)

        return None

    def portfolio_density(data, geofile):
        #Portfolio Density Plots
        st.title('Region Overview')

        c1, c2 = st.beta_columns((1, 1))
        c1.header('Portfolio Density')

        df = data.sample(10)

        # Base map - Folium
        density_map = folium.Map(location=[data['lat'].mean(), data['long'].mean()],
                                 default_zoom_start=15)

        marker_cluster = MarkerCluster().add_to(density_map)

        for name, row in df.iterrows():
            folium.Marker([row['lat'], row['long']],
                          popup='Price R${0} on: {1}, Features: {2} sqft, {3} bedrooms, {4} bathrooms, {5} year_built'.format(
                              row['price'],
                              row['date'],
                              row['sqft_living'],
                              row['bedrooms'],
                              row['bathrooms'],
                              row['year_built'])).add_to(marker_cluster)

        with c1:
            folium_static(density_map)

        # Region Price Map
        c2.header('Price Density')

        df = data[['price', 'zipcode']].groupby('zipcode').mean().reset_index()
        df.colums = ['ZIP', 'PRICE']

        df = df.sample(10)

        geofile = geofile[geofile['ZIP'].isin(df['ZIP'].tolist())]

        region_price_map = folium.Map(location=[data['lat'].mean(), data['long'].mean()],
                                      default_zoom_start=15)

        region_price_map.choropleth(data=df,
                                    geo_data=geofiles,
                                    columns=['ZIP', 'PRICE'],
                                    key_on='feature.properties.ZIP',
                                    fill_color='YlOrRd',
                                    fill_opacity=0.7,
                                    line_opacity=0.2,
                                    legend_name='AVG PRICE')

        with c2:
            folium_static(region_price_map)

        return None

    def commercial_distribution(data):
        #Houses distribution
        st.sidebar.title('Commercial Options')
        st.title('Commercial Attributes')

        # filters
        data['date'] = pd.to_datetime(data['date'])
        min_year_built = int(data['yr_built'].min())
        max_year_built = int(data['yr_built'].max())

        st.sidebar.subheader('Select Max Year Built')
        f_year_built = st.sidebar.slider('Year Built', min_year_built,
                                         max_year_built, min_year_built)

        st.header('Average price per Year Built')
        # ---------------Average Price per Year

        df = data.loc[data['yr_built'] < f_year_built]
        df = data[['yr_built', 'price']].groupby('yr_built').mean().reset_index()

        fig = px.line(df, x='yr_built', y='price')

        st.plotly_chart(fig, use_container_width=True)

        # ---------------Average Price per Day
        st.header('Average Price per day')
        st.sidebar.subheader('Select Max Date')

        # filter
        min_date = datetime.strptime(data['date'].min(), '%Y-%m-%d')
        max_date = datetime.strptime(data['date'].max(), '%Y-%m-%d')

        f_date = st.sidebar.slider('Date', min_date,
                                   max_date, min_date)

        # data filtering
        data['date'] = pd.to_datetime(data['date'])
        df = data.loc[data['date'] < f_date]
        df = data[['date', 'price']].groupby('yr_built').mean().reset_index()

        # plot
        fig = px.line(df, x='date', y='price')
        st.plotly_chart(fig, use_container_width=True)

        # ------------------------------------
        # Histograma
        st.header('Price Distribution')
        st.sidebar.subheader('Select Max Price')

        # filter
        min_price = int(data['price'].min())
        max_price = int(data['price'].max())
        avg_price = int(data['price'].mean())

        # filter
        f_price = st.sidebar.slider('Price', min_price,
                                    max_price, avg_price)

        # data filtering
        df = data.loc[data['price'] < f_price]

        # plot
        fig = px.histogram(df, x='price', nbins=50)
        st.plotly_chart(fig, use_container_width=True)

        return None

    def attributes_distribution(data):
        #all attributes
        st.sidebar.title('Attributes Options')
        st.title('House Attributes')

        # filters
        f_bedrooms = st.sidebar.selectbox('Max Number Bedrooms',
                                          sorted(set(data['bedrooms'].unique())))

        f_bathrooms = st.sidebar.selectbox('Max Number Bathrooms',
                                           sorted(set(data['bathrooms'].unique())))

        c1, c2 = st.beta_columns(2)

        # Houses per bedrooms
        c1.header('Houses per bedrooms')
        df = data[data['bedrooms'] < f_bedrooms]
        fig = px.histogram(df, x='bedrooms', nbins=19)
        c1.plotly_chart(fig, use_container_width=True)

        # Houses per bathrooms
        c2.header('Houses per bathrooms')
        df = data[data['bathrooms'] < f_bathrooms]
        fig = px.histogram(df, x='bathrooms', nbins=19)
        c2.plotly_chart(fig, use_container_width=True)

        # filter
        f_floors = st.sidebar.selectbox('Max number of floor',
                                        sorted(set(data['floors'].unique())))

        c1, c2 = st.beta_columns(2)
        # Houses per floors
        c1.header('Houses per floor')
        df = data[data['floors'] < f_floors]

        # plot
        fig = px.histogram(df, x='floors', nbins=19)
        c1.plotly_chart(fig, use_container_width=True)

        # House per water view
        c2.header('Houses per water view')
        if f_waterview:
            df = data[data['waterfront'] == 1]

        else:
            df = data.copy()

        # plot
        fig = px.histogram(df, x='waterfront', nbins=10)
        c2.plotly_chart(fig, use_container_width=True)

        return None

In [ ]:
streamlit_settings()

# EXTRACTION

In [ ]:
#getting the original dataframe
path = 'C:\\Users\\joaoa\\Documents\\DSprojects\\Git\\repos\\DataScience_Em_Producao\\House_Rocket\\kc_house_data.csv'
data = get_data(path)

# get geofile
path2 = 'C:\\Users\\joaoa\\Documents\\DSprojects\\Git\\repos\\DataScience_Em_Producao\\House_Rocket\\Zip_Codes.geojson'
geofile = get_geofile(url)

#original url for .json download
url = 'https://opendata.arcgis.com/datasets/83fc2e72903343aabff6de8cb445b81c_2.geojson'

# TRANSFORMATION

In [ ]:
#transformation
data = set_feature(data)

overview_data(data)

portfolio_density(data, geofile)

commercial_distribution(data)

attributes_distribution(data)

# LOADING

In [ ]:
if __name__ == '__main__':
   
    #ETL
    #extraction

    #transformation
    
    #loading

#------------------------------------------------------------------------------------------------------------------------------

# 0.0 IMPORTS

In [6]:
#libs
import pandas as pd
import numpy as np
import folium
import streamlit as st
from folium.plugins import MarkerCluster
import plotly.express as px
from datetime import datetime
import geopandas
from streamlit_folium import folium_static
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings

## 0.1 Helper Functions 

In [7]:
#functions    
    
def settings():
        plt.style.use('bmh')
        plt.rcParams['figure.figsize']=21,12
        plt.rcParams['font.size']=24
    
        warnings.filterwarnings('ignore')#to ignore that we're dividing by zero
    
        sns.set()
        
def streamlit_settings():
        st.set_page_config(layout='wide')
        st.cache(allow_output_mutation=True)
        return None

def get_data(path):
        data = pd.read_csv(path)
        return data

st.cache(allow_output_mutation=True)
def get_geofile(url):
        geofile = geopandas.read_file(url)
        return geofile

def set_feature(data):
        #add new features
        data['price_m2'] = data['price']/data['sqft_lot']
        return data

def overview_data(data):
        #Data Overview
        f_attributes = st.sidebar.multiselect('Enter columns', data.columns)
        f_zipcode = st.sidebar.multiselect('Enter zipcode',
                                           data['zipcode'].unique())
        st.title('Data Overview')

        if (f_zipcode != []) & (f_attributes != []):
            data = data.loc[data['zipcode'].isin(f_zipcode), f_attributes]

        elif (f_zipcode != []) & (f_attributes == []):
            data = data.loc[data['zipcode'].isin(f_zipcode), :]

        elif (f_zipcode == []) & (f_attributes != []):
            data = data.loc[:, f_attributes]

        else:
            data = data.copy()

        st.dataframe(data)

        c1, c2 = st.beta_columns((2, 1))

        # average metrics:
        df1 = data[['id', 'zipcode']].groupby('zipcode').count().reset_index()
        df2 = data[['price', 'zipcode']].groupby('zipcode').mean().reset_index()
        df3 = data[['sqft_living', 'zipcode']].groupby('zipcode').mean().reset_index()
        df4 = data[['price_m2', 'zipcode']].groupby('zipcode').mean().reset_index()

        # merge
        m1 = pd.merge(df1, df2, on='zipcode', how='inner')
        m2 = pd.merge(m1, df3, on='zipcode', how='inner')
        m3 = pd.merge(m2, df4, on='zipcode', how='inner')

        m3.columns = ['zipcode', 'total houses', 'price', 'sqft living', 'price/m2']

        c1.header('Average Values')
        c1.dataframe(m3, height=600)

        # descriptive stats
        num_attributes = data.select_dtypes(['int64', 'float64'])
        media = pd.DataFrame(num_attributes.apply(np.mean))
        mediana = pd.DataFrame(num_attributes.apply(np.median))
        std = pd.DataFrame(num_attributes.apply(np.std))
        max_ = pd.DataFrame(num_attributes.apply(np.max))
        min_ = pd.DataFrame(num_attributes.apply(np.min))

        df1 = pd.concat([max_, min_, media, mediana, std], axis=1).reset_index()

        df1.columns = ['attributes', 'max', 'min', 'mean', 'median', 'std']

        c2.header('Descriptive Statistics')
        c2.dataframe(df1, height=800)

        st.write(data.head())
        # attributes => colunas
        # zipcode => linhas

        st.write(f_attributes)
        st.write(f_zipcode)

        return None

def portfolio_density(data, geofile):
        #Portfolio Density Plots
        st.title('Region Overview')

        c1, c2 = st.beta_columns((1, 1))
        c1.header('Portfolio Density')

        df = data.sample(10)

        # Base map - Folium
        density_map = folium.Map(location=[data['lat'].mean(), data['long'].mean()],
                                 default_zoom_start=15)

        marker_cluster = MarkerCluster().add_to(density_map)

        for name, row in df.iterrows():
            folium.Marker([row['lat'], row['long']],
                          popup='Price R${0} on: {1}, Features: {2} sqft, {3} bedrooms, {4} bathrooms, {5} year_built'.format(
                              row['price'],
                              row['date'],
                              row['sqft_living'],
                              row['bedrooms'],
                              row['bathrooms'],
                              row['year_built'])).add_to(marker_cluster)

        with c1:
            folium_static(density_map)

        # Region Price Map
        c2.header('Price Density')

        df = data[['price', 'zipcode']].groupby('zipcode').mean().reset_index()
        df.colums = ['ZIP', 'PRICE']

        df = df.sample(10)

        geofile = geofile[geofile['ZIP'].isin(df['ZIP'].tolist())]

        region_price_map = folium.Map(location=[data['lat'].mean(), data['long'].mean()],
                                      default_zoom_start=15)

        region_price_map.choropleth(data=df,
                                    geo_data=geofiles,
                                    columns=['ZIP', 'PRICE'],
                                    key_on='feature.properties.ZIP',
                                    fill_color='YlOrRd',
                                    fill_opacity=0.7,
                                    line_opacity=0.2,
                                    legend_name='AVG PRICE')

        with c2:
            folium_static(region_price_map)

        return None

def commercial_distribution(data):
        #Houses distribution
        st.sidebar.title('Commercial Options')
        st.title('Commercial Attributes')

        # filters
        data['date'] = pd.to_datetime(data['date'])
        min_year_built = int(data['yr_built'].min())
        max_year_built = int(data['yr_built'].max())

        st.sidebar.subheader('Select Max Year Built')
        f_year_built = st.sidebar.slider('Year Built', min_year_built,
                                         max_year_built, min_year_built)

        st.header('Average price per Year Built')
        # ---------------Average Price per Year

        df = data.loc[data['yr_built'] < f_year_built]
        df = data[['yr_built', 'price']].groupby('yr_built').mean().reset_index()

        fig = px.line(df, x='yr_built', y='price')

        st.plotly_chart(fig, use_container_width=True)

        # ---------------Average Price per Day
        st.header('Average Price per day')
        st.sidebar.subheader('Select Max Date')

        # filter
        min_date = datetime.strptime(data['date'].min(), '%Y-%m-%d')
        max_date = datetime.strptime(data['date'].max(), '%Y-%m-%d')

        f_date = st.sidebar.slider('Date', min_date,
                                   max_date, min_date)

        # data filtering
        data['date'] = pd.to_datetime(data['date'])
        df = data.loc[data['date'] < f_date]
        df = data[['date', 'price']].groupby('yr_built').mean().reset_index()

        # plot
        fig = px.line(df, x='date', y='price')
        st.plotly_chart(fig, use_container_width=True)

        # ------------------------------------
        # Histograma
        st.header('Price Distribution')
        st.sidebar.subheader('Select Max Price')

        # filter
        min_price = int(data['price'].min())
        max_price = int(data['price'].max())
        avg_price = int(data['price'].mean())

        # filter
        f_price = st.sidebar.slider('Price', min_price,
                                    max_price, avg_price)

        # data filtering
        df = data.loc[data['price'] < f_price]

        # plot
        fig = px.histogram(df, x='price', nbins=50)
        st.plotly_chart(fig, use_container_width=True)

        return None

def attributes_distribution(data):
        #all attributes
        st.sidebar.title('Attributes Options')
        st.title('House Attributes')

        # filters
        f_bedrooms = st.sidebar.selectbox('Max Number Bedrooms',
                                          sorted(set(data['bedrooms'].unique())))

        f_bathrooms = st.sidebar.selectbox('Max Number Bathrooms',
                                           sorted(set(data['bathrooms'].unique())))

        c1, c2 = st.beta_columns(2)

        # Houses per bedrooms
        c1.header('Houses per bedrooms')
        df = data[data['bedrooms'] < f_bedrooms]
        fig = px.histogram(df, x='bedrooms', nbins=19)
        c1.plotly_chart(fig, use_container_width=True)

        # Houses per bathrooms
        c2.header('Houses per bathrooms')
        df = data[data['bathrooms'] < f_bathrooms]
        fig = px.histogram(df, x='bathrooms', nbins=19)
        c2.plotly_chart(fig, use_container_width=True)

        # filter
        f_floors = st.sidebar.selectbox('Max number of floor',
                                        sorted(set(data['floors'].unique())))

        c1, c2 = st.beta_columns(2)
        # Houses per floors
        c1.header('Houses per floor')
        df = data[data['floors'] < f_floors]

        # plot
        fig = px.histogram(df, x='floors', nbins=19)
        c1.plotly_chart(fig, use_container_width=True)

        # House per water view
        c2.header('Houses per water view')
        if f_waterview:
            df = data[data['waterfront'] == 1]

        else:
            df = data.copy()

        # plot
        fig = px.histogram(df, x='waterfront', nbins=10)
        c2.plotly_chart(fig, use_container_width=True)

        return None

In [8]:
streamlit_settings()
settings()

## 0.2 Loading Data

In [ ]:
#getting the original dataframe
path = 'C:\\Users\\joaoa\\Documents\\DSprojects\\Git\\repos\\DataScience_Em_Producao\\House_Rocket\\kc_house_data.csv'
df_raw = get_data(path)

# get geofile
path2 = 'C:\\Users\\joaoa\\Documents\\DSprojects\\Git\\repos\\DataScience_Em_Producao\\House_Rocket\\Zip_Codes.geojson'
url = 'https://opendata.arcgis.com/datasets/83fc2e72903343aabff6de8cb445b81c_2.geojson'
geofile = get_geofile(url)

df_raw.head(3)

# 1.0 Data Description

## 1.1 Rename Columns

## 1.2 Data Dimension

## 1.3 Data Types

## 1.4 Check NAs

## 1.5 Fillout NA

## 1.6 Change Types

## 1.7 Descriptive Statistics

# 2.0 FEATURE ENGINEERING

## 2.1 Hypothesis Creation

## 2.2 Final List of Hypothesis

## Feature Engineering

# 3.0 EXPLORATORY DATA ANALYSIS

## 3.1 Univariate Analysis

## 3.2 Bivariate Analysis

## Multivariate Analysis

In [5]:
t = (1, 2, 3, [4, 5, 6])

In [6]:
t[3][0] = 7
t

(1, 2, 3, [7, 5, 6])

In [2]:
l = [1,2,3]

In [8]:
l

[1, 2, (1, 2, 3, [7, 5, 6])]

In [3]:
l.reduce()

AttributeError: 'list' object has no attribute 'reduce'